# Timer

In [5]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 10, 22, 13, 30

while True:
    dt = datetime.now()
    if dt.month >= month and dt.day >= day:
        if dt.hour > hour or (dt.hour == hour and dt.minute > minute):
            break
    sleep(60)

# Parameter optimization

In [6]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['PriceChange']
work_timeframe = '15m'
higher_timeframe = '4h'
opt_limit = 50
load = False

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {'PriceChange': {"low_price_quantile": [1 * (i + 1) for i in range(20)]}}

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['indicator_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat = stat.sort_values(['indicator_rank', 'pct_right_forecast_avg'], ascending=False)
stat.head(20)

Number of combinations is 20


  0%|          | 0/20 [00:00<?, ?it/s]

,pattern,PriceChange_low_price_quantile,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,indicator_rank
0,PriceChange,1,81.64,68.4,60.46,58.87,64.17,67.34,67.96,61.17,58.96,62.84,65.67,66.11,65.05,62.4,60.46,59.58,59.58,60.46,60.46,57.63,56.31,56.49,55.34,54.72,-0.16,-0.14,-0.1,-0.12,-0.21,-0.23,-0.28,-0.19,-0.16,-0.23,-0.26,-0.31,-0.36,-0.35,-0.35,-0.36,-0.38,-0.38,-0.37,-0.21,-0.2,-0.19,-0.15,-0.15,1133,62.169583,-8871.862083
1,PriceChange,2,81.64,68.4,60.46,58.87,64.17,67.34,67.96,61.17,58.96,62.84,65.67,66.11,65.05,62.4,60.46,59.58,59.58,60.46,60.46,57.63,56.31,56.49,55.34,54.72,-0.16,-0.14,-0.1,-0.12,-0.21,-0.23,-0.28,-0.19,-0.16,-0.23,-0.26,-0.31,-0.36,-0.35,-0.35,-0.36,-0.38,-0.38,-0.37,-0.21,-0.2,-0.19,-0.15,-0.15,1133,62.169583,-8871.862083
2,PriceChange,3,81.64,68.4,60.46,58.87,64.17,67.34,67.96,61.17,58.96,62.84,65.67,66.11,65.05,62.4,60.46,59.58,59.58,60.46,60.46,57.63,56.31,56.49,55.34,54.72,-0.16,-0.14,-0.1,-0.12,-0.21,-0.23,-0.28,-0.19,-0.16,-0.23,-0.26,-0.31,-0.36,-0.35,-0.35,-0.36,-0.38,-0.38,-0.37,-0.21,-0.2,-0.19,-0.15,-0.15,1133,62.169583,-8871.862083
3,PriceChange,4,81.64,68.4,60.46,58.87,64.17,67.34,67.96,61.17,58.96,62.84,65.67,66.11,65.05,62.4,60.46,59.58,59.58,60.46,60.46,57.63,56.31,56.49,55.34,54.72,-0.16,-0.14,-0.1,-0.12,-0.21,-0.23,-0.28,-0.19,-0.16,-0.23,-0.26,-0.31,-0.36,-0.35,-0.35,-0.36,-0.38,-0.38,-0.37,-0.21,-0.2,-0.19,-0.15,-0.15,1133,62.169583,-8871.862083
4,PriceChange,5,81.64,68.4,60.46,58.87,64.17,67.34,67.96,61.17,58.96,62.84,65.67,66.11,65.05,62.4,60.46,59.58,59.58,60.46,60.46,57.63,56.31,56.49,55.34,54.72,-0.16,-0.14,-0.1,-0.12,-0.21,-0.23,-0.28,-0.19,-0.16,-0.23,-0.26,-0.31,-0.36,-0.35,-0.35,-0.36,-0.38,-0.38,-0.37,-0.21,-0.2,-0.19,-0.15,-0.15,1133,62.169583,-8871.862083
5,PriceChange,6,81.64,68.4,60.46,58.87,64.17,67.34,67.96,61.17,58.96,62.84,65.67,66.11,65.05,62.4,60.46,59.58,59.58,60.46,60.46,57.63,56.31,56.49,55.34,54.72,-0.16,-0.14,-0.1,-0.12,-0.21,-0.23,-0.28,-0.19,-0.16,-0.23,-0.26,-0.31,-0.36,-0.35,-0.35,-0.36,-0.38,-0.38,-0.37,-0.21,-0.2,-0.19,-0.15,-0.15,1133,62.169583,-8871.862083
6,PriceChange,7,81.64,68.4,60.46,58.87,64.17,67.34,67.96,61.17,58.96,62.84,65.67,66.11,65.05,62.4,60.46,59.58,59.58,60.46,60.46,57.63,56.31,56.49,55.34,54.72,-0.16,-0.14,-0.1,-0.12,-0.21,-0.23,-0.28,-0.19,-0.16,-0.23,-0.26,-0.31,-0.36,-0.35,-0.35,-0.36,-0.38,-0.38,-0.37,-0.21,-0.2,-0.19,-0.15,-0.15,1133,62.169583,-8871.862083
7,PriceChange,8,81.64,68.4,60.46,58.87,64.17,67.34,67.96,61.17,58.96,62.84,65.67,66.11,65.05,62.4,60.46,59.58,59.58,60.46,60.46,57.63,56.31,56.49,55.34,54.72,-0.16,-0.14,-0.1,-0.12,-0.21,-0.23,-0.28,-0.19,-0.16,-0.23,-0.26,-0.31,-0.36,-0.35,-0.35,-0.36,-0.38,-0.38,-0.37,-0.21,-0.2,-0.19,-0.15,-0.15,1133,62.169583,-8871.862083
8,PriceChange,9,81.64,68.4,60.46,58.87,64.17,67.34,67.96,61.17,58.96,62.84,65.67,66.11,65.05,62.4,60.46,59.58,59.58,60.46,60.46,57.63,56.31,56.49,55.34,54.72,-0.16,-0.14,-0.1,-0.12,-0.21,-0.23,-0.28,-0.19,-0.16,-0.23,-0.26,-0.31,-0.36,-0.35,-0.35,-0.36,-0.38,-0.38,-0.37,-0.21,-0.2,-0.19,-0.15,-0.15,1133,62.169583,-8871.862083
9,PriceChange,

# Save new config data to config file

In [4]:
from config_updater import ConfigUpdater

timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'PriceChange': {"low_price_quantile": [12]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)